### API Yahoo Finance

In [1]:
# previamente se  instala las siguientes librerias "yfinance".

In [2]:
# librería para descargar con yahoo finance
import yfinance as yf
# librería para manipular dataframes
import pandas as pd
# librería para manipular fechas
import datetime as dt

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [3]:
# Diccionario con las empresas de las que se traen los datos
dict_Empresas = {
    "Apple": "AAPL",
    "Microsoft": "MSFT",
    "Tesla": "TSLA",
    "Amazon" : "AMZN",
    "Alphabet": "GOOGL",


    # Algunas sugerencias:
    
    #"JPMorgan": "JPM",
    #"Johnson & Johnson": "JNJ",
    #"Walmart": "WMT"
    }

In [4]:
import os

# Crear el directorio Datasets si no existe
if not os.path.exists('Datasets'):
    os.makedirs('Datasets')

# Bucle para descargar y guardar datos
for key, value in dict_Empresas.items():
    # Se descargan los ultimos 5 años.
    Dataset = yf.download(value, start=dt.datetime.now() - dt.timedelta(days=5*365), end=dt.date.today())
    
    Dataset = Dataset[["Close", "Volume"]]
    # Dataset["Ticker"] = value

    # Se guardan en un archivo con el nombre de la empresa
    Dataset.to_csv(f"Datasets/{key}.csv")

    # Se imprime en pantalla un mensaje de confirmación
    print(f"Archivo {key} cargado exitosamente.")


[*********************100%%**********************]  1 of 1 completed


Archivo Apple cargado exitosamente.


[*********************100%%**********************]  1 of 1 completed


Archivo Microsoft cargado exitosamente.


[*********************100%%**********************]  1 of 1 completed


Archivo Tesla cargado exitosamente.


[*********************100%%**********************]  1 of 1 completed


Archivo Amazon cargado exitosamente.


[*********************100%%**********************]  1 of 1 completed

Archivo Alphabet cargado exitosamente.


### SQL

In [5]:
import sqlite3

# Conexión a la base de datos
conn = sqlite3.connect('./Database/test.db')
conn.close()

In [6]:
Dataset
#Que es?

,Close,Volume
Date,,
2019-05-29,55.997002,36230000
2019-05-30,56.070499,18088000
2019-05-31,55.325001,31582000
2019-06-03,51.937000,96890000
2019-06-04,52.724499,60512000
...,...,...
2024-05-20,176.919998,22554400
2024-05-21,177.850006,16989400
2024-05-22,176.380005,17880000


In [7]:
import os

# Carpeta con los archivos CSV
folder_path = './Datasets'

# Obtener la lista de archivos en la carpeta
files = os.listdir(folder_path)

files

['Amazon.csv', 'Alphabet.csv', 'Apple.csv', 'Tesla.csv', 'Microsoft.csv']

In [8]:
# Lista para almacenar los DataFrames de cada archivo CSV
dfs = []

# Leer cada archivo CSV en un DataFrame y agregarlo a la lista
for file in files:
    if file.endswith('.csv'):
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path)
        df['Empresa'] = file.split('.')[0]
        dfs.append(df)

# Concatenar los DataFrames en uno solo
result = pd.concat(dfs, ignore_index=True)

# Imprimir el resultado
result.head()

,Date,Close,Volume,Empresa
0,2019-05-29,90.959503,85580000,Amazon
1,2019-05-30,90.816002,62938000,Amazon
2,2019-05-31,88.753502,92376000,Amazon
3,2019-06-03,84.634499,181974000,Amazon
4,2019-06-04,86.477997,113582000,Amazon


In [9]:
DB = './Database/stock_db.db'
conn = sqlite3.connect(DB)
print("Connected to SQLite")
conn.close()


Connected to SQLite


In [10]:
DB = './Database/stock_db.db'
conn = sqlite3.connect(DB)
print("Connected to SQLite")
result.to_sql('main', conn, schema=None, 
              if_exists='append')
conn.close()

Connected to SQLite


In [11]:
DB = './Database/stock_db.db'
conn = sqlite3.connect(DB)
print("Connected to SQLite")
result.to_sql('main', conn, schema=None, 
              if_exists='replace')

Connected to SQLite


6290

In [12]:
# Creo una función que haga todo:

def write_sql_main(DB, df, table):
	"""Esta funcion inserta masivamente un df o una lista de tuplas en SQL."""
	try:
		conn = sqlite3.connect(DB)
		print("Connected to SQLite")
		df.to_sql(table, conn, schema=None, if_exists='append', index=False, index_label=None, chunksize=None, dtype=None, method=None)
		result = 'ok'
		df.to_sql(table, conn, schema=None, if_exists='replace', index=False, index_label=None, chunksize=None, dtype=None, method=None)
	except sqlite3.Error as error:
		print("Failed to insert multiple records into sqlite table", error)
		result = 'failed'
	finally:
		if conn:
			conn.close()
			print("The SQLite connection is closed")
	return result

In [13]:
write_sql_main(DB=DB, df = result, table = 'main2')

Connected to SQLite
The SQLite connection is closed


'ok'

In [14]:
write_sql_main(DB=DB, df = result, table = 'trabajo_practico_yhf')

Connected to SQLite


The SQLite connection is closed


'ok'